TASK 1


*PART 1: Acceptance status*

In [1]:
#Import libraries
import numpy as numpy
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [2]:
#Loading file
from google.colab import files
uploaded = files.upload()

Saving processed_reviews_split_surnamesIJKLM_minimal (1).csv to processed_reviews_split_surnamesIJKLM_minimal (1).csv


In [3]:
df = pd.read_csv('processed_reviews_split_surnamesIJKLM_minimal (1).csv')

df.head(5)

,review_id,text,confidence_score,review_score,acceptance_status
0,iclr_review_0000,Discriminative k-shot learning using probabili...,3.0,5.0,Reject
1,iclr_review_0001,FastText.zip: Compressing text classification ...,3.0,6.0,Reject
2,iclr_review_0002,ADAPTIVE GENERATION OF PROGRAMMING PUZZLES. AI...,NaN,3.0,Reject
3,iclr_review_0003,Learning Graph Convolution Filters from Data M...,4.0,6.0,Reject
4,iclr_review_0004,wMAN: WEAKLY-SUPERVISED MOMENT ALIGNMENT NETWO...,NaN,6.0,Reject


In [4]:
df.shape

(6118, 5)

In [5]:
df.describe()

,confidence_score,review_score
count,3301.000000,6118.000000
mean,3.803999,4.686172
std,0.822079,2.164158
min,1.000000,-1.000000
25%,3.000000,3.000000
50%,4.000000,5.000000
75%,4.000000,6.000000
max,5.000000,10.000000


In [6]:
#Checking for duplicates and removing them

df.drop_duplicates(inplace=True)

In [7]:
df.shape

(6118, 5)

In [9]:
## Checking for missing data and dropping them

df.isnull().sum()

review_id               0
text                   84
confidence_score     2817
review_score            0
acceptance_status     392
dtype: int64

In [10]:
df = df.dropna()

In [11]:
df.shape

(3260, 5)

In [12]:
## Downloading stopword package
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
def text_processing(text):
  # Punctutaion removal
  nopunc = [char for char in text if char not in string.punctuation]
  nopunc = ''.join(nopunc)

  # Stopwords removal
  cleanwords = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
  return cleanwords 


In [15]:
df['text'].head().apply(text_processing)

0     [Discriminative, kshot, learning, using, proba...
1     [FastTextzip, Compressing, text, classificatio...
3     [Learning, Graph, Convolution, Filters, Data, ...
5     [Metacontrol, Adaptive, ImaginationBased, Opti...
11    [Latent, Convolutional, Models, present, new, ...
Name: text, dtype: object

In [16]:
## Generating dummy variables for "acceptance_status"
data_new = pd.get_dummies(df, columns=['acceptance_status'])
data_new.head()

,review_id,text,confidence_score,review_score,acceptance_status_Accept,acceptance_status_Reject
0,iclr_review_0000,Discriminative k-shot learning using probabili...,3.0,5.0,0,1
1,iclr_review_0001,FastText.zip: Compressing text classification ...,3.0,6.0,0,1
3,iclr_review_0003,Learning Graph Convolution Filters from Data M...,4.0,6.0,0,1
5,iclr_review_0005,Metacontrol for Adaptive Imagination-Based Opt...,3.0,7.0,1,0
11,iclr_review_0011,Latent Convolutional Models. We present a new ...,2.0,7.0,1,0


In [17]:
data_new= data_new.drop(['acceptance_status_Reject'], axis = 1)

In [18]:
data_new

,review_id,text,confidence_score,review_score,acceptance_status_Accept
0,iclr_review_0000,Discriminative k-shot learning using probabili...,3.0,5.0,0
1,iclr_review_0001,FastText.zip: Compressing text classification ...,3.0,6.0,0
3,iclr_review_0003,Learning Graph Convolution Filters from Data M...,4.0,6.0,0
5,iclr_review_0005,Metacontrol for Adaptive Imagination-Based Opt...,3.0,7.0,1
11,iclr_review_0011,Latent Convolutional Models. We present a new ...,2.0,7.0,1
...,...,...,...,...,...
6103,iclr_review_6103,BEHAVIOR MODULE IN NEURAL NETWORKS. Prefrontal...,5.0,4.0,0
6107,iclr_review_6107,A Teacher Student Network For Faster Video Cla...,5.0,4.0,0
6112,iclr_review_6112,Skip-graph: Learning graph embeddings with an ...,4.0,5.0,0
6113,iclr_review_6113,"Universality, Robustness, and Detectability of...",3.0,6.0,0


In [19]:
## Converting message corpus into tokens
from sklearn.feature_extraction.text import CountVectorizer
message_bow = CountVectorizer(analyzer=text_processing).fit_transform(data_new['text']) 

In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(message_bow, data_new['acceptance_status_Accept'], test_size=0.33, random_state=0)

In [22]:
## Developing classification models using Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)

In [23]:
# Printing the predictions
print(clf.predict(X_train))

# Printing actual values
print(y_train.values)

[0 1 1 ... 0 1 1]
[0 1 1 ... 0 1 1]


In [24]:
# Evaluating model on training dataset
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = clf.predict(X_test)
print(classification_report(y_test, pred))
print()
print('Confusion matrix: \n', confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

           0       0.78      0.87      0.82       642
           1       0.77      0.64      0.70       434

    accuracy                           0.78      1076
   macro avg       0.77      0.75      0.76      1076
weighted avg       0.77      0.78      0.77      1076


Confusion matrix: 
 [[557  85]
 [157 277]]


In [25]:
print('Accuracy: ', accuracy_score(y_test, pred))

Accuracy:  0.775092936802974


PART 2: Review score

In [26]:
X_train2,X_test2,y_train2,y_test2 = train_test_split(message_bow, data_new['review_score'], test_size=0.33, random_state=0)

In [28]:
## Developing classification models using Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf2 = MultinomialNB().fit(X_train2, y_train2)

In [29]:
pred2 = clf2.predict(X_test2)
print(classification_report(y_test2, pred2))
print()
print('Confusion matrix: \n', confusion_matrix(y_test2, pred2))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00        27
         1.0       0.00      0.00      0.00         4
         2.0       0.00      0.00      0.00        18
         3.0       0.27      0.03      0.06        95
         4.0       0.28      0.33      0.30       203
         5.0       0.28      0.32      0.30       236
         6.0       0.28      0.47      0.35       225
         7.0       0.30      0.26      0.28       185
         8.0       0.25      0.02      0.03        64
         9.0       0.00      0.00      0.00        15
        10.0       0.00      0.00      0.00         4

    accuracy                           0.28      1076
   macro avg       0.15      0.13      0.12      1076
weighted avg       0.26      0.28      0.25      1076


Confusion matrix: 
 [[  0   0   0   2   6   7   7   5   0   0   0]
 [  0   0   0   0   1   3   0   0   0   0   0]
 [  0   0   0   1   5   4   6   2   0   0   0]
 [  0   0   0   3  38  21

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
print('Accuracy: ', accuracy_score(y_test2, pred2))

Accuracy:  0.2815985130111524


In [31]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier().fit(X_train2, y_train2)

In [32]:
pred_dt = tree.predict(X_test2)
print(classification_report(y_test2, pred_dt))
print()
print('Confusion matrix: \n', confusion_matrix(y_test2, pred_dt))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00        27
         1.0       0.00      0.00      0.00         4
         2.0       0.00      0.00      0.00        18
         3.0       0.23      0.19      0.21        95
         4.0       0.30      0.31      0.31       203
         5.0       0.26      0.22      0.24       236
         6.0       0.27      0.32      0.30       225
         7.0       0.21      0.21      0.21       185
         8.0       0.11      0.11      0.11        64
         9.0       0.08      0.07      0.07        15
        10.0       0.00      0.00      0.00         4

    accuracy                           0.23      1076
   macro avg       0.13      0.13      0.13      1076
weighted avg       0.23      0.23      0.23      1076


Confusion matrix: 
 [[ 0  0  0  4  4  5  9  4  1  0  0]
 [ 0  0  0  1  1  0  2  0  0  0  0]
 [ 0  0  0  2  6  3  3  3  0  1  0]
 [ 5  0  2 18 22 17 13 11  6  0  1]
 [ 8  0  3 13 63 33 34

In [33]:
print('Accuracy: ', accuracy_score(y_test2, pred_dt))

Accuracy:  0.23327137546468402
